In [1]:
import pandas as pd
import numpy as np

# 数据归一化操作
def normalize_feature(df):
    return df.apply(lambda column: (column - column.mean()) / column.std())


df = normalize_feature(pd.read_csv('data1.csv',
                                   names=['square', 'bedrooms', 'price']))

ones = pd.DataFrame({'ones': np.ones(len(df))})# 生成一行ones，其是n行1列的数据框，表示x0恒为1
df = pd.concat([ones, df], axis=1)  # 根据列将ones数据和我们从data1文件中读取出的数据合并

#将x(输入数据)和y(输出数据)进行分离，用于训练
X_data = np.array(df[df.columns[0:3]]) 
y_data = np.array(df[df.columns[-1]]).reshape(len(df), 1)

print(X_data.shape, type(X_data))
print(y_data.shape, type(y_data))

(47, 3) <class 'numpy.ndarray'>
(47, 1) <class 'numpy.ndarray'>


## 创建线性回归模型（数据流图）

__get_variable方法和variable方法相比有什么好处呢？__

在训练的时候我们使用get_variable方法可以直接获取到我们当前数据流图中名为你设置的名称这个变量的值，以上图为例，就是我们获取名为weight的变量的值。
也就是说在我们进行反复训练的时候，我们可能会从我们的checkpoint中获取我们的之前的模型参数，他可以快速的帮我们找到我们模型参数的值，当我们要加载我们的模型参数到数据流图中时，我们就可以通过我们的key:value的格式找到我们对应参数的值了。对我们进行参数的赋值加载有一个很好的作用。 同时也支持我们initializer初始化。


In [11]:
import tensorflow as tf

alpha = 0.01 # 学习率 alpha
epoch = 500 # 训练全量数据集的轮数

#首先我们定义我们的输入数据和输出数据的模型(数据流图)，因为我们是在会话中训练数据的时候才把我们的输入数据和输出数据填充进数据流图，
#所以我们先用占位符创建一个数据类型和数据形状即可，在经过我们的归一化操作之后他是一个浮点数，所以我们使用浮点数定义，
#同时定义我们的X(输入数据的形状为[47,3]，根据我们之前的数据形状定义) y也同理。
with tf.name_scope('input'):
    # 输入 X，形状[47, 3]
    X = tf.placeholder(tf.float32, X_data.shape, name='X')
    # 输出 y，形状[47, 1]
    y = tf.placeholder(tf.float32, y_data.shape, name='y')

#定义我们的权重变量，因为我们的假设函数为个 h(x) = w0*x0+w1*x1+w2*x2，而且从我们的输入数据X中可以看到变量数量是3个，
#所以我们W的形状应该是[3,1] ，所以我们代码中(X_data.shape[1], 1)表示的就是我们的行数和X中的列数保持一致也就是3，同时将列数设置为1 ,
#W就是我们常用公式hθ(x)=θ^Tx中θ的转置。此时我们在计算我们的假设函数hθ(x)时我们就可以直接进行我们的矩阵相乘 hθ(x)=tf.matmul(X, W)
       
with tf.name_scope('hypothesis'):
    # 权重变量 W，形状[3,1]
    W = tf.get_variable("weights",
            (X_data.shape[1], 1),initializer=tf.constant_initializer())
    # 假设函数 h(x) = w0*x0+w1*x1+w2*x2, 其中x0恒为1
    # 推理值 y_pred  形状[47,1]
    print(W)
    #之前我们有说个hθ(x)=θ^Tx，这里的W等于θ^Tx，所以求假设函数hθ(x)时我们就可以直接进行我们的矩阵相乘 hθ(x)=tf.matmul(X, W)
    #就是我们的[47,3]这个形状的矩阵乘以[3,1]这个形状的矩阵。最后我们的到的就是[47,1] 47行1列的预测值

    y_pred = tf.matmul(X, W, name='y_pred')

with tf.name_scope('loss'):
    # 损失函数采用最小二乘法，y_pred - y 是形如[47, 1]的向量。
    # tf.matmul(a,b,transpose_a=True) 表示：矩阵a的转置乘矩阵b，即 [1,47] X [47,1] transpose_a=True 表示转置a矩阵
    # 损失函数操作 loss
    loss_op = 1 / (2 * len(X_data)) * tf.matmul((y_pred - y), (y_pred - y), transpose_a=True)
with tf.name_scope('train'):
    # 随机梯度下降优化器 opt
    #这里使用梯度下降的方法进行优化，这条代码定义了一个随机梯度优化器，同时我们给优化器传入一个learning_rate=alpha 也就是我们的学习率，这时候我们就定义好了一个优化器opt。
    #然后我们通过train_op=opt. minimize(loss_op)，我们通过传入我们的损失函数，通过minimize不断的进行最小化。这个时候我们得到的模型参数就是我们想要的模型参数。
    train_op = tf.train.GradientDescentOptimizer(learning_rate=alpha).minimize(loss_op)

ValueError: Variable weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\ops.py", line 1801, in __init__
    self._traceback = tf_stack.extract_stack()
  File "D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\framework\ops.py", line 3300, in create_op
    op_def=op_def)
  File "D:\APYPLA~1\venv_dir\venv1\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)


## 创建会话（运行环境）

In [3]:
#如果要对我们的模型参数进行训练我们需要创建一个会话。
with tf.Session() as sess:
    # 初始化全局变量
    sess.run(tf.global_variables_initializer())
    # 开始训练模型
    # 因为训练集较小，所以每轮都使用全量数据训练
    #表示我们训练epoch轮我们的数据，一轮代表一次数据流图的训练，也就是我们一次epoch就是训练我们的47组数据，当我们进行一次全量数据集的训练，我们就可以理解为我们进行了一次epoch。
    for e in range(1, epoch + 1):
        sess.run(train_op, feed_dict={X: X_data, y: y_data})
        if e % 10 == 0:
            loss, w = sess.run([loss_op, W], feed_dict={X: X_data, y: y_data})
            log_str = "Epoch %d \t Loss=%.4g \t Model: y = %.4gx1 + %.4gx2 + %.4g"
            print(log_str % (e, loss, w[1], w[2], w[0]))
       

Epoch 10 	 Loss=0.4116 	 Model: y = 0.0791x1 + 0.03948x2 + 3.353e-10
Epoch 20 	 Loss=0.353 	 Model: y = 0.1489x1 + 0.07135x2 + -5.588e-11
Epoch 30 	 Loss=0.3087 	 Model: y = 0.2107x1 + 0.09676x2 + 3.912e-10
Epoch 40 	 Loss=0.2748 	 Model: y = 0.2655x1 + 0.1167x2 + -1.863e-11
Epoch 50 	 Loss=0.2489 	 Model: y = 0.3142x1 + 0.1321x2 + 1.77e-10
Epoch 60 	 Loss=0.2288 	 Model: y = 0.3576x1 + 0.1436x2 + -4.47e-10
Epoch 70 	 Loss=0.2131 	 Model: y = 0.3965x1 + 0.1519x2 + -8.103e-10
Epoch 80 	 Loss=0.2007 	 Model: y = 0.4313x1 + 0.1574x2 + -6.985e-10
Epoch 90 	 Loss=0.1908 	 Model: y = 0.4626x1 + 0.1607x2 + -4.936e-10
Epoch 100 	 Loss=0.1828 	 Model: y = 0.4909x1 + 0.1621x2 + -6.147e-10
Epoch 110 	 Loss=0.1763 	 Model: y = 0.5165x1 + 0.162x2 + -7.87e-10
Epoch 120 	 Loss=0.1709 	 Model: y = 0.5397x1 + 0.1606x2 + -5.821e-10
Epoch 130 	 Loss=0.1664 	 Model: y = 0.5609x1 + 0.1581x2 + -9.08e-10
Epoch 140 	 Loss=0.1625 	 Model: y = 0.5802x1 + 0.1549x2 + -9.965e-10
Epoch 150 	 Loss=0.1592 	 Model: y 